In [3]:
import numpy as np

import matplotlib.pyplot as plt

In [19]:
directory = "./results/least_squares"

methods = [
    ('S-SZD' , f'{directory}/sszd.log'),
    ('SZVR-G' , f'{directory}/szvr_g.log'),
    # ('Spider-SZO' , f'{directory}/spider_szo.log'),
    # ('ZO-Spider-Coord' , f'{directory}/zo_spider_coord.log'),
    ('ZO-SVRG-Ave' , f'{directory}/zosvrg_ave.log'),
    ('ZO-SVRG-Coord' , f'{directory}/zosvrg_coord.log'),
    ('ZO-SVRG-Coord-Rand' , f'{directory}/zosvrg_coord_rand.log'),
    ('O-SVRZ' , f'{directory}/osvrz.log'),
]
budget = 1000000

def read_result(budget, path):
    mu_val, std_val = [], []
    mu_tim, std_tim = [], []
    with open(path, 'r') as f:
        for line in f.readlines():
            splitted = line.split(",")
            mu_val += [float(splitted[0]) for _ in range(int(splitted[-1]))]
            std_val += [float(splitted[1]) for _ in range(int(splitted[-1]))]
            mu_tim += [float(splitted[2]) for _ in range(int(splitted[-1]))]
            std_tim += [float(splitted[3]) for _ in range(int(splitted[-1]))]
            if len(mu_val) > budget:
                break
    mu_val, std_val = np.array(mu_val)[:budget], np.array(std_val)[:budget]
    mu_tim, std_tim = np.array(mu_tim)[:budget], np.array(std_tim)[:budget]
    return dict(values=(mu_val, std_val), times=(mu_tim, std_tim))


In [21]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.set_title("Function values")
ax2.set_title("Cumulative Computational Cost")

for (label, path) in methods:
    ris = read_result(budget, path)
    mu_values, std_values = ris['values']
    mu_times,  std_times  = ris['times']
    
    ax1.plot(range(len(mu_values)), mu_values, '-', label=label)
    ax1.fill_between(range(len(mu_values)), mu_values - std_values, mu_values + std_values, alpha=0.6)
    ax2.plot(range(len(mu_times)), mu_times, '-', label=label)
    ax2.fill_between(range(len(mu_times)), mu_times - std_times, mu_times + std_times, alpha=0.6)
    ris = None

ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.legend(loc='upper right')

ax1.set_xlabel("# stochastic function values", fontsize=12)
ax1.set_ylabel("$F(x^\\tau) - F(x^*)$", fontsize=12)

ax2.set_xlabel("# stochastic function values", fontsize=12)
ax2.set_ylabel("Cost (s)", fontsize=12)

fig.tight_layout()
fig.savefig("test.png", bbox_inches='tight')
